In [1]:
import ipywidgets as widgets
import pandas as pd
import joblib
import requests

# Load your pre-trained random forest model
model = joblib.load('randomforest.joblib')
scaler = joblib.load('scaler.joblib')
feature_names = joblib.load('feature_names.joblib')

# Create input widgets
num_rooms = widgets.FloatSlider(description='Rooms', min=1, max=10, step=0.5, value=3)
floor = widgets.IntSlider(description='Floor', min=0, max=20, value=5)
living_space = widgets.IntSlider(description='Living Space (sqft)', min=10, max=500, value=100)
street_input = widgets.Text(description='Street Name')
city_input = widgets.Text(description='City')
postal_code_input = widgets.Text(description='Postal Code')

# Create OK button
ok_button = widgets.Button(description='OK')

# Function to geocode address using API
def geocode_address(street, city, postal_code):
    url = f"https://nominatim.openstreetmap.org/search?street={street}&city={city}&postalcode={postal_code}&country=Switzerland&format=json"
    response = requests.get(url)
    data = response.json()
    if data:
        latitude = data[0]['lat']
        longitude = data[0]['lon']
        return latitude, longitude
    else:
        return None, None

def predict_price(button):
    # Get user inputs
    latitude, longitude = geocode_address(street_input.value, city_input.value, postal_code_input.value)
    if latitude is None or longitude is None:
        print("Unable to geocode address.")
        return
    features = pd.DataFrame({
        'Rooms': [num_rooms.value],
        'Floor': [floor.value],
        'Living Space (sqft)': [living_space.value],
        'Latitude': [latitude],
        'Longitude': [longitude],
        'PostalCode': [postal_code_input.value], 
    })
    
    # Standardize the features
    features = scaler.transform(features.values)
    
    # Predict price
    predicted_price = model.predict(features)[0]
    
    # Calculate the standard deviation of the predictions from each tree
    predictions = [tree.predict(features) for tree in model.estimators_]
    std_dev = np.std(predictions)
    
    # Display predicted price and confidence interval
    with price_output:
        print(f'Predicted Price: CHF {predicted_price:.2f}')
        print(f'Confidence Interval: CHF {predicted_price - 2*std_dev:.2f} - CHF {predicted_price + 2*std_dev:.2f}')
    
    # Display feature importances
    importances = model.feature_importances_
    for feature_name, importance in zip(feature_names, importances):
        print(f'{feature_name}: {importance*100:.2f}%')

# Attach event handler to OK button
ok_button.on_click(predict_price)

# Create output widget for displaying predicted price
price_output = widgets.Output()

# Display widgets
widgets.VBox([num_rooms, floor, living_space, street_input, city_input, postal_code_input, ok_button, price_output])

c:\Users\User\anaconda3\envs\adspenv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


AttributeError: 'GridSearchCV' object has no attribute 'estimators_'